<a href="https://colab.research.google.com/github/MatteoOnger/VIA_Project/blob/main/VIA_EQM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VIA Project: EqMotion Meets Assembly101 - Equivariant Motion Modeling for Manipulation Tasks**

*   **Author:** Matteo Onger
*   **Date:** September 2025

**Documentation**:
*   Dataset: [Assembly101](https://assembly-101.github.io/)
*   Model: [EqMotion](https://github.com/MediaBrain-SJTU/EqMotion)
*   Training Framework: [4DHands](https://github.com/FrancescoAgnelli3/4D_hands)

**Notes**:
*   For faster execution, please use a GPU-equipped runtime.

## Clone the GitHub repository

In [ ]:
# check PyTorch and CUDA version
import torch
print(torch.__version__)

In [ ]:
# install dependencies
!pip install torchcde
!pip install torch-dct
!pip install torch-geometric

!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html       # set PyTorch and CUDA version
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html        # set PyTorch and CUDA version

!pip install git+https://github.com/TorchSpatiotemporal/tsl.git

In [1]:
# clone training framework
!git clone https://github.com/FrancescoAgnelli3/4D_hands

Cloning into '4D_hands'...
remote: Enumerating objects: 862, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 862 (delta 19), reused 35 (delta 11), pack-reused 802 (from 1)
Receiving objects: 100% (862/862), 43.18 MiB | 36.36 MiB/s, done.
Resolving deltas: 100% (337/337), done.


## Load the dataset

### From SFTP server

In [ ]:
from google.colab import userdata

In [ ]:
# set the credentials and paths
local_path = '/content'
data_folder = 'data_our'
remote_path = '/datasets/AssemblyHands/assembly101-download-scripts/data_our'

host = userdata.get('phuse_server')     # server domain must be SET in SECRETS
user = userdata.get('phuse_user')       # server user must be SET in SECRETS
psw = userdata.get('phuse_psw')         # user password must be SET in SECRETS

user_at_host = user + '@' + host
data_path = local_path + '/' + data_folder

In [ ]:
# install sshpass
!sudo apt update
!sudo apt install sshpass

# create directories
!mkdir -p $remote_path
!mkdir -p ~/.ssh

# add server to known hosts
!ssh-keyscan $host >> ~/.ssh/known_hosts

# download the data
!sshpass -p $psw sftp -oBatchMode=no -b - $user_at_host <<< "get -r {remote_path} {local_path}"

### From drive

In [2]:
from google.colab import drive

In [3]:
# mount Google Drive
drive.mount('/content/drive')

# set the paths
data_path = '/content/drive/MyDrive/Colab Notebooks/UNIMI/VIA Project/data_our'

Mounted at /content/drive


### Mount the dataset

In [4]:
import os

In [8]:
#  mount the dataset
expected_path = '/content/4D_hands/Assembly/data'

if not os.path.exists(expected_path):
    os.symlink(data_path, expected_path)
    print(f'Symlink created: {expected_path} -> {data_path}')
else:
    print(f'Symlink or folder already exists: {expected_path}')

Symlink created: /content/4D_hands/Assembly/data -> /content/drive/MyDrive/Colab Notebooks/UNIMI/VIA Project/data_our


## Run the models

In [9]:
#  set working directory
%cd /content/4D_hands/Assembly/spatiotemporal

/content/4D_hands/Assembly/spatiotemporal


In [11]:
!python /content/4D_hands/Assembly/spatiotemporal/test_prediction.py \
    --dataset Assembly \
    --action rotate \
    --models gmn EGNN \
    --batch_size 64 \
    --time 100 \
    --window 90

Number of model (gmn_att) parameters:      41780
Epoch [1/50] Train Loss: 0.107990 | Test Loss: 0.050792 | LR: 0.001000
Epoch [2/50] Train Loss: 0.095343 | Test Loss: 0.058069 | LR: 0.001000
Epoch [3/50] Train Loss: 0.106409 | Test Loss: 0.049135 | LR: 0.001000
Epoch [4/50] Train Loss: 0.095280 | Test Loss: 0.053957 | LR: 0.001000
Epoch [5/50] Train Loss: 0.099059 | Test Loss: 0.048324 | LR: 0.001000
Epoch [6/50] Train Loss: 0.087262 | Test Loss: 0.051922 | LR: 0.001000
Epoch [7/50] Train Loss: 0.095468 | Test Loss: 0.047726 | LR: 0.001000
Epoch [8/50] Train Loss: 0.089848 | Test Loss: 0.051313 | LR: 0.001000
Epoch [9/50] Train Loss: 0.088400 | Test Loss: 0.049314 | LR: 0.001000
Epoch [10/50] Train Loss: 0.088022 | Test Loss: 0.047720 | LR: 0.000500
Epoch [11/50] Train Loss: 0.091792 | Test Loss: 0.047651 | LR: 0.000500
Epoch [12/50] Train Loss: 0.088911 | Test Loss: 0.047028 | LR: 0.000500
Epoch [13/50] Train Loss: 0.085841 | Test Loss: 0.048077 | LR: 0.000500
Epoch [14/50] Train Loss